In [33]:
%matplotlib inline

import glob
import os
import re
import pandas as pd
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

import skimage.transform
import scipy.ndimage
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
import tensorflow as tf

import scipy.misc
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

DATA_PATH = '/kaggle_2/luna/luna16/data/original_lungs/'
OUTPUT_FOLDER = '/kaggle_2/luna/luna16/data/pre_processed_chunks_augmented_v2/'
ANNOTATIONS_PATH = '/kaggle_2/luna/luna16/data/original_lungs/csv/annotations.csv'
ANNOTATIONS_EXCLUDED_PATH = '/kaggle_2/luna/luna16/data/original_lungs/csv/annotations_excluded.csv'
CANDIDATES_PATH = '/kaggle_2/luna/luna16/data/original_lungs/csv/candidates.csv'
CHUNK_SIZE = 48
CANDIDATES_SIZE = 10000
NUM_CLASSES = 4

In [34]:
annotations = pd.read_csv(ANNOTATIONS_PATH)
annotations_excluded = pd.read_csv(ANNOTATIONS_EXCLUDED_PATH)
candidates = pd.read_csv(CANDIDATES_PATH)
candidates_sampled = candidates[candidates['class'] == 0].sample(CANDIDATES_SIZE)
candidates_sampled

,seriesuid,coordX,coordY,coordZ,class
90776,1.3.6.1.4.1.14519.5.2.1.6279.6001.148229375703...,-159.050000,-171.910000,-348.690000,0
137177,1.3.6.1.4.1.14519.5.2.1.6279.6001.174907798609...,-23.020000,-16.110000,-226.690000,0
410474,1.3.6.1.4.1.14519.5.2.1.6279.6001.334166493392...,-106.300000,-138.440000,-302.920000,0
402408,1.3.6.1.4.1.14519.5.2.1.6279.6001.329326052298...,32.923779,60.106004,-208.690090,0
483499,1.3.6.1.4.1.14519.5.2.1.6279.6001.663019255629...,73.200000,168.160000,-565.900000,0
200853,1.3.6.1.4.1.14519.5.2.1.6279.6001.208511362832...,51.330000,-43.580000,-66.380000,0
97862,1.3.6.1.4.1.14519.5.2.1.6279.6001.152706273988...,31.009322,38.049252,-160.163032,0
65404,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-36.370000,-117.110000,-164.870000,0
159439,1.3.6.1.4.1.14519.5.2.1.6279.6001.187694838527...,-65.510000,-142.720000,-174.935000,0
399264,1.3.6.1.4.1.14519.5.2.1.6279.6001.325580698241...,-39.405921,-51.776974,-103.196579,0


In [35]:
def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)
    
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    
    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))
    
    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))
    
    return ct_scan, origin, spacing

In [36]:
def get_label(diameter_mm):
    if int(diameter_mm) == -1:
        return 0
    elif (diameter_mm >= 0.0) and (diameter_mm < 6.0):
        return 1
    elif (diameter_mm >= 6.0) and (diameter_mm < 12.0):
        return 2
    elif diameter_mm >= 12.0:     
        return 3

all_annotations = pd.concat([annotations_excluded, annotations], ignore_index=True)
all_annotations['class'] = all_annotations.diameter_mm.apply(get_label)
all_annotations = all_annotations.drop('diameter_mm', 1)

train_annotations = pd.concat([all_annotations, candidates_sampled], ignore_index=True)

In [37]:
train_annotations['class'].value_counts()

0    40513
1     2924
2     2098
3      843
Name: class, dtype: int64

In [46]:
def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

def voxel_2_world(voxel_coordinates, origin, spacing):
    stretched_voxel_coordinates = voxel_coordinates * spacing
    world_coordinates = stretched_voxel_coordinates + origin
    return world_coordinates

In [39]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

In [40]:
def plot_2d(im):
    # Show some slice in the middle
    plt.imshow(im, cmap=plt.cm.bone)
    plt.show()
    
def plot_3d(image, threshold=-300):
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces = measure.marching_cubes(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()

In [25]:
patient_annotations = train_annotations[train_annotations.seriesuid == '1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893536422914241295628']
patient_annotations

,seriesuid,coordX,coordY,coordZ,class
4005,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-65.058519,-252.394456,-166.300000,0
4006,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,52.480632,-144.152186,-92.500000,0
4007,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-31.855369,-129.542800,-117.700000,0
4008,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-31.855369,-126.886549,-114.100000,0
4009,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-41.152251,-95.011524,-142.900000,0
4010,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-39.824124,-184.660029,-292.300000,0
4011,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-112.871054,-235.128817,-322.900000,0
4012,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-86.972597,-185.324093,-117.700000,0
4013,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-65.058519,-252.394456,-166.300000,0
4014,1.3.6.1.4.1.14519.5.2.1.6279.6001.137375498893...,-65.722581,-252.394456,-177.100000,0


In [55]:
patient_uids = train_annotations.seriesuid.unique()

patients_processed_files = glob.glob(OUTPUT_FOLDER + '[0-9\.]*_X.npy')
patients_processed = set()
for filename in patients_processed_files:
    m = re.match(r'([0-9\.]*)_X.npy', os.path.basename(filename))
    patients_processed.add(m.group(1))

print(len(patient_uids))

graph = tf.Graph()
with graph.as_default():
    img_chunk = tf.placeholder(tf.float32, shape=[CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE], name='img_chunk')
    img_flipped_up_down = tf.image.flip_up_down(img_chunk)
    img_flipped_left_right = tf.image.flip_left_right(img_chunk)
    img_rot_90 = tf.image.rot90(img_chunk)
    img_trans = tf.image.transpose_image(img_chunk)

weird_chunks = {}

for patient_uid in patient_uids:
    if patient_uid in patients_processed:
        print('Skipping already processed patient {}'.format(patient_uid))
        continue
    print('Processing patient {}'.format(patient_uid))
    
    patient_annotations = train_annotations[train_annotations.seriesuid == patient_uid]
    patient_scans_path = glob.glob(DATA_PATH + 'subset?/{}.mhd'.format(patient_uid))[0]
    img, origin, spacing = load_itk(patient_scans_path)

    #print('Origin ', origin)
    #print('Scan shape ', img.shape)
    #print('Spacing ', spacing)
    
    #calculate resize factor
    RESIZE_SPACING = [1, 1, 1]
    resize_factor = spacing / RESIZE_SPACING
    new_real_shape = img.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize = new_shape / img.shape
    new_spacing = spacing / real_resize
    
    #print('real_resize ', real_resize)
    # Resample; resize image to 1mmx1mmx1mm spacing
    lung_img = scipy.ndimage.interpolation.zoom(img, real_resize)
    
    #new_origin = voxel_2_world([-upper_offset,-lower_offset,0],origin,new_spacing)
    #print('Resampled shape ', lung_img.shape)
    #print('New spacing ', new_spacing)
    
    num_class_not_0 = patient_annotations[patient_annotations['class'] != 0].shape[0]
    num_class_0 = patient_annotations.shape[0] - num_class_not_0
      
    count = 0
    X = np.ndarray([num_class_0 + num_class_not_0 * 5, CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE], dtype=np.int16)
    Y = np.ndarray([num_class_0 + num_class_not_0 * 5, NUM_CLASSES], dtype=np.int16)
    for annotation in patient_annotations.itertuples():
        y = annotation[5]
        coordX = annotation[2]
        coordY = annotation[3]
        coordZ = annotation[4]
        imageCoord = np.array((coordZ, coordY, coordX))
        
        #print('imageCoord ', imageCoord, y)
        # Convert coords to voxel coords and slice lung_img
        imageCoord = world_2_voxel(imageCoord, origin, RESIZE_SPACING)
        
        coordX1 = int(imageCoord[2]) - int(CHUNK_SIZE/2)
        coordX2 = int(imageCoord[2]) + int(CHUNK_SIZE/2)
        coordY1 = int(imageCoord[1]) - int(CHUNK_SIZE/2)
        coordY2 = int(imageCoord[1]) + int(CHUNK_SIZE/2)
        coordZ1 = int(imageCoord[0]) - int(CHUNK_SIZE/2)
        coordZ2 = int(imageCoord[0]) + int(CHUNK_SIZE/2)
        
        coordX1 = 0 if (coordX1 < 0) else coordX1
        coordY1 = 0 if (coordY1 < 0) else coordY1
        coordZ1 = 0 if (coordZ1 < 0) else coordZ1
        
        coordX2 = lung_img.shape[2] if (coordX2 > lung_img.shape[2]) else coordX2
        coordY2 = lung_img.shape[1] if (coordY2 > lung_img.shape[1]) else coordY2
        coordZ2 = lung_img.shape[0] if (coordZ2 > lung_img.shape[0]) else coordZ2
      
        if (coordZ2 < coordZ1 or coordY2 < coordY1 or coordX2 < coordX1):
            print('Foudn weird chunk!')
            if patient_uid in weird_chunks.keys():
                weird_chunks[patient_uid].append({'shape': lung_img.shape, 'class': y, 'center_coords': imageCoord, 'chunk_coords': [coordZ1, coordZ2, coordY1, coordY2, coordX1, coordX2]})
            else:
                weird_chunks[patient_uid] = []
                weird_chunks[patient_uid].append({'shape': lung_img.shape, 'class': y, 'center_coords': imageCoord, 'chunk_coords': [coordZ1, coordZ2, coordY1, coordY2, coordX1, coordX2]})
            continue
        #print('imageCoord ', imageCoord)
        #print('chunk ', coordZ1, coordZ2, coordY1, coordY2, coordX1, coordX2)
        chunk = np.full((CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), -1000.0, np.float32)
        chunk[0:coordZ2-coordZ1, 0:coordY2-coordY1, 0:coordX2-coordX1] = lung_img[coordZ1:coordZ2,coordY1:coordY2,coordX1:coordX2]

        X[count,:,:,:] = chunk
        Y[count,] = (np.arange(NUM_CLASSES) == y)+0
        count = count + 1
        
        # Augment non-zero classes (cancerous nodules)
        if y != 0:
            with tf.Session(graph=graph) as session:
                session.run(tf.global_variables_initializer())
                chunk_flipped_up_down, chunk_flipped_left_right, chunk_rot_90, chunk_trans = session.run([img_flipped_up_down, 
                                                                                                          img_flipped_left_right,
                                                                                                          img_rot_90,
                                                                                                          img_trans], feed_dict={img_chunk: chunk})
                
                X[count,:,:,:] = chunk_flipped_up_down
                Y[count,] = (np.arange(NUM_CLASSES) == y)+0
                count = count + 1

                X[count,:,:,:] = chunk_flipped_left_right
                Y[count,] = (np.arange(NUM_CLASSES) == y)+0
                count = count + 1

                X[count,:,:,:] = chunk_rot_90
                Y[count,] = (np.arange(NUM_CLASSES) == y)+0
                count = count + 1

                X[count,:,:,:] = chunk_trans
                Y[count,] = (np.arange(NUM_CLASSES) == y)+0
                count = count + 1
  
    np.save(OUTPUT_FOLDER + patient_uid + '_X.npy', X)
    np.save(OUTPUT_FOLDER + patient_uid + '_Y.npy', Y)
    
for k, v in weird_chunks.items():
    print('patient_id ', k, ' ', v)

888
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840553388986847034053
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100530488926682752765845212286
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100620385482151095585000946543
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100684836163890911914061745866
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028192176989979435275
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.101228986346984399347858840086
Skipping already processed patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.102133688497886810253331438

/kaggle/dev/ashish/assignment-2/assignment-2/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.283733738239331719775105586296
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.283878926524838648426928238498
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.285926554490515269336267972830
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.286061375572911414226912429210
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.286217539434358186648717203667
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.286422846896797433168187085942
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.286627485198831346082954437212
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.286647622786041008124419915089
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.287560874054243719452635194040
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.288701997968615460794642979503
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.29013515687409836642487